# Criando o jogo

Nessa simplificação, o jogo do pong é criado com o campo sendo um array do `numpy` e havendo apenas uma raquete (que será nosso NPC), pois vamos considerar que do outro lado as bolas são sempre rebatidas.

In [1]:
from math import floor
import numpy as np
import random as rd

class PongGame():
    """
    Jogo do Pong

    O jogo consiste em uma barra que se move horizontalmente na parte inferior 
    da tela, e uma bola que se move verticalmente e horizontalmente. O objetivo 
    é fazer a bola bater na barra e voltar para cima, sem deixar a bola cair na 
    parte inferior da tela. A cada vez que a bola bate na barra, o jogador ganha 
    um ponto. O jogo termina quando a bola cai na parte inferior da tela.

    O jogo é representado por uma matriz de 60x100, onde 0 representa uma 
    posição vazia, 1 representa a posição da barra e 2 representa a posição da 
    bola.
    """
    def __init__(self):
        """
        Inicializa o jogo do Pong
        """
        self.board_length = 60 # Tamanho do tabuleiro
        self.board_height = 100 # Altura do tabuleiro
        self.board = np.zeros((self.board_length, self.board_height)) # Tabuleiro
        self.player = 1 # Representação do jogador
        self.player_position = 0 # Posição inicial do jogador
        self.ball_position = np.array([ # Posição inicial da bola
            rd.randint(int(self.board_length / 6), int(self.board_length * 5 / 6)), 
            rd.randint(int(self.board_height / 10), int(self.board_height / 2))],
            dtype="float64")
        self.ball_direction = np.array([ # Direção inicial da bola
            rd.randint(int(-self.board_length / 12), int(self.board_length / 12)), 
            1],
            dtype="float64")
        self._normalize_direction() # Normaliza a direção da bola
        self.player_length = int(self.board_length/6) # Tamanho do jogador
        self.score = 0 # Pontuação do jogador
        self.winner = False # Indica se o jogador ganhou a rodada
        self.game_over = False # Indica se o jogo acabou

    def _move_player(self, move):
        """
        Move o jogador para a esquerda (-1), direita (1) ou não se move (0)

        Parâmetros
        ----------
        move : int
            Direção para a qual o jogador deve se mover
        """
        if self.player_position + move >= 0 \
            and self.player_position + move + self.player_length <= self.board_length:
            self.player_position += move

    def _normalize_direction(self):
        """
        Normaliza a direção da bola para que ela sempre tenha a mesma velocidade

        A velocidade da bola é definida como 1
        """
        norm = np.linalg.norm(self.ball_direction)
        self.ball_direction = self.ball_direction / norm

    def _move_ball(self):
        """
        Move a bola para a próxima posição
        """
        # Atualiza a posição da bola
        self.ball_position[0] += self.ball_direction[0]
        self.ball_position[1] += self.ball_direction[1]
        # Corrigir a posição da bola caso ela tenha saído do tabuleiro
        if self.ball_position[0] < 0:
            self.ball_position[0] *= -1
            self.ball_direction[0] *= -1
        if self.ball_position[0] >= self.board_length:
            self.ball_position[0] = 2 * self.board_length - self.ball_position[0] - 1
            self.ball_direction[0] *= -1
        if self.ball_position[1] >= self.board_height:
            self.ball_position[1] = 2 * self.board_height - self.ball_position[1] - 1
            self.ball_direction[1] *= -1
        # Verifica se a bola bateu na barra
        if self.ball_position[1] < 1:
            if floor(self.ball_position[0]) >= self.player_position and floor(self.ball_position[0]) <= self.player_position + self.player_length:
                self.ball_position[1] = 2 - self.ball_position[1]
                self.ball_direction = self.ball_position - np.array([self.player_position + (self.player_length/2), 0])
                self._normalize_direction()
                self.score += 1
                self.winner = True
            else:
                self.winner = False
                self.game_over = True

    def play(self, move):
        """
        Executa uma jogada

        Parâmetros
        ----------
        move : int
            Direção para a qual o jogador deve se mover

        Retorno
        -------
        winner : bool
            True se o jogador ganhou a rodada, False caso contrário
        """
        self._move_player(move)
        self._move_ball()
        return self.winner

    def print_board(self):
        """
        Imprime o tabuleiro do jogo
        """
        self.board = np.zeros((self.board_length, self.board_height))
        self.board[self.player_position: self.player_position + self.player_length, 0] = self.player
        self.board[floor(self.ball_position[0]), floor(self.ball_position[1])] = 2
        print(self.board)

# Algoritmos Genético

## Algoritmo base

Utilizamos de base uma [implementação em python do algoritmo genético](https://asimov.academy/como-programar-um-algoritmo-de-otimizacao-genetica/) já publicada, como segue abaixo:

In [2]:
import pandas as pd
from copy import deepcopy
import random

class GeneticOptimizer:
    def __init__(self, functions):
        self.functions = {
            'initialize_individual':None,
            'boundaries':None,
            'mutation':None,
            'fitness': None,
        }
        self._init_functions(functions)
        
        self.final_results = {
            'max_fitness': None,
            'individual_max': None
        }
        self.population = []
        self.generation = 0
        
    def _init_functions(self, functions):
        for func_type in self.functions:
            if func_type not in functions.keys():
                print(func_type, 'não fornecido.')
        for func_type in functions:
            self.functions[func_type] = functions[func_type]
    
    def _initialize_population(self):
        while len(self.population) < self.n_population:
            self.population.append(self.functions['initialize_individual']())
            
    def _apply_boundaries(self):
        self.population = self.functions['boundaries'](self.population)
            
    def _evaluate(self):
        self.list_fitness = self.functions['fitness'](self.population)
        df = pd.DataFrame({'individual':self.population, 'fitness': self.list_fitness}).sort_values('fitness', ascending=False)
        self.population = df['individual'].to_list()
        self.list_fitness = df['fitness'].to_list()
        individual_max = self.population[0]
        fitness_max = self.list_fitness[0]
        self._all_time_best(individual_max, fitness_max)
        
    def _all_time_best(self, individual_max, fitness_max):
        if self.final_results['max_fitness'] == None:
            self.final_results['individual_max'] = individual_max
            self.final_results['max_fitness'] = fitness_max
        elif fitness_max > self.final_results['max_fitness']:
            self.final_results['individual_max'] = individual_max
            self.final_results['max_fitness'] = fitness_max
        print(f"[ GeneticOptimizer ] Geração {self.generation} --- Melhor fitness: {fitness_max} --- Campeã: {self.final_results['max_fitness']}")

    def _cut_individuals(self):
        self.population = self.population[:self.n_keep_individuals]
        
    def _mate(self):
        combinations = []
        for i in range(len(self.population)):
            for j in range(i+1,len(self.population)):
                combinations.append([i, j])
        combinations = combinations[:self.n_children]
        for couple in combinations:
            new_individual = deepcopy(self.population[couple[0]])
            for count, parameter in enumerate(new_individual):
                if random.random() > 0.5:
                    new_individual[count] = self.population[couple[1]][count]
            new_individual = self.functions['mutation'](new_individual, self.mutation_probability)
            self.population.append(new_individual)
    
    def _cycle(self):
        self._apply_boundaries()
        self._evaluate()
        self._cut_individuals()
        self._mate()
        self._initialize_population()
        
        self.generation += 1
        
    def optimize(self, n_population=100, n_keep_individuals=10, n_children=15, mutation_probability=0.01, generations=100):
        self.n_population = n_population
        self.n_keep_individuals = n_keep_individuals
        self.n_children = n_children
        self.mutation_probability = mutation_probability
        self.generations = generations
        
        self._initialize_population()
        for i in range(generations):
            self._cycle()
            
        print(f"[ GeneticOptimizer ] Otimização finalizada --- Campeão: {self.final_results['max_fitness']}")
        


## Definindo as funções iniciais
Com o algoritmo de base, definimos abaixo as funções iniciais para utilizá-lo:

In [3]:
def initialize_individual():
    """
    Função que inicializa um indivíduo.

    Returns
    -------
    individual : list
        Lista com os parâmetros do indivíduo.
    """
    individual = []
    for i in range(9):
        individual.append(random.randint(-10, 10))
    return individual

def boundaries(population):
    """
    Função que aplica as restrições aos indivíduos.

    Parameters
    ----------
    population : list
        Lista com os indivíduos da população.

    Returns
    -------
    population : list
        Lista com os indivíduos da população com as restrições aplicadas.

    """
    return population

def mutation(individual, mutation_probability):
    """
    Função que aplica a mutação em um indivíduo.

    Parameters
    ----------
    individual : list
        Lista com os parâmetros do indivíduo.
    mutation_probability : float
        Probabilidade de mutação.

    Returns
    -------
    individual : list
        Lista com os parâmetros do indivíduo com a mutação aplicada.

    """
    new_individual = deepcopy(individual)
    for count, parameter in enumerate(individual):
        if random.random() < mutation_probability:
            new_individual[count] = random.randint(-10, 10)
    return new_individual

def fitness(population):
    """
    Função que calcula o fitness de cada indivíduo.

    Parameters
    ----------
    population : list
        Lista com os indivíduos da população.

    Returns
    -------
    list_fitness : list
        Lista com os fitness de cada indivíduo.

    """
    list_fitness = [] # Lista com os fitness de cada indivíduo
    for individual in population: # Para cada indivíduo
        game = PongGame() # Inicializa o jogo
        count = 0 # Contador de ações
        while not game.game_over and count < 100000: # Enquanto o jogo não acabar e o contador for menor que 100000
            count += 1 # Incrementa o contador
            x = game.ball_position[0] - game.player_position # Calcula a diferença entre a posição da bola e a posição do jogador
            y = game.ball_position[1] # Calcula a posição da bola no eixo y
            w = individual # Parâmetros do indivíduo
            # Calcula a ação do indivíduo
            move = w[0]*x + w[1]*y + w[2]*x*y + w[3]*x**2 + w[4]*y**2 
            move += w[5]*x**2*y + w[6]*x*y**2 + w[7] + w[8]*x**2*y**2 
            move = np.sign(move) 
            # Executa a ação do indivíduo
            game.play(move) 
        fitness = game.score # Fitness do indivíduo
        list_fitness.append(fitness) # Adiciona o fitness do indivíduo na lista
    return list_fitness # Retorna a lista com os fitness de cada indivíduo

Com isso, podemos treinar o modelo:

In [7]:
# Dicionário com as funções que serão utilizadas pelo algoritmo genético
functions = {
    'initialize_individual':initialize_individual,
    'boundaries':boundaries,
    'mutation':mutation,
    'fitness': fitness,
}

# Inicializa o algoritmo genético
optimizer = GeneticOptimizer(functions)

# Executa a otimização
optimizer.optimize(n_population=100, n_children=100, n_keep_individuals=10,generations=100)


Vendo o resultado do melhor indivíduo, obtivemos (os coeficientes):

In [45]:
# Imprime o resultado final da otimização (indivíduo com o maior fitness)
best_individual = optimizer.final_results['individual_max']
print(best_individual)

[-3, -10, -8, 1, -7, 2, -10, -3, 2]


Testando o melhor indivíduo com mais iterações:

In [51]:
game = PongGame() # Inicializa o jogo
count = 0 # Contador de ações
# Enquanto o jogo não acabar e o contador for menor que 10000000
while not game.game_over and count < 10000000:
    # Realiza a jogada de acordo com os coeficientes indivíduo com o maior 
    # fitness e a posição atual da bola
    x = game.ball_position[0] - game.player_position
    y = game.ball_position[1]
    w = best_individual
    move = w[0]*x + w[1]*y + w[2]*x*y + w[3]*x**2 + w[4]*y**2
    move += w[5]*x**2*y + w[6]*x*y**2 + w[7] + w[8]*x**2*y**2
    move = np.sign(move)
    game.play(move)
    count += 1
# Imprime o resultado final do jogo
print("Movimentos:", count)
print("Pontuação:", game.score)

Movimentos: 10000000
Pontuação: 38711


Assim, vemos que o algoritmo funcionou como esperado, pois além de convergir rapidamente de forma a obter um indivíduo que complete todas as iterações na função de _fitness_, também testamos esse indivíduo com cem vezes mais iterações e ele continua sem perder.

# Referências

GMEA. Como programar algoritmo de Otimização Genética. 2021. <[https://asimov.academy/como-programar-um-algoritmo-de-otimizacao-genetica/](https://asimov.academy/como-programar-um-algoritmo-de-otimizacao-genetica/)>.